## Metropolis Hastings Algorithm

In [32]:
import pandas as pd
import numpy as np
import math
import pymc3 as pm

The dataset is from CSSS-STAT 564 U Washington

In [16]:
df = pd.read_csv('./../data/sparrow.csv',usecols=['age2','fledged'])
df =  df.rename(columns = {
    'age2':'x','fledged':'y'
})
df = df[['x','y']]
df.columns

Index(['x', 'y'], dtype='object')

In [17]:
data = df.values

## Poisson distribution : z<sup>k</sup>e<sup>-z</sup> / k!

Assumption : 
### A Poisson regression model can be fitted to find how x and y is related. 
### Poisson regression model , here is a GLM.

y ~ f (b<sub>1</sub> + b<sub>1</sub>x<sup>2</sup> + b<sub>1</sub>x<sub>2</sub>)

More specifically,    
#### log(E(y<sub>i</sub>| x<sub>i</sub>)) = b<sub>1</sub> + b<sub>1</sub>x<sup>2</sup> + b<sub>1</sub>x<sub>2</sub>


Prior distribution : b<sub>1</sub>, b<sub>2</sub>, b<sub>3</sub> are i.i.d and ~ N(0,10)

###  What can be the choice of generating the next parameter value, given current value being z ?
 - It should be symmetric
 - Uniform or Normal works well. U(z-d, u+d) or N(z, d<sup>2</sup>)
 - d controls " how far to deviate from current"
 - Too small a d, burn in period may be high due to algorithmm being stuck
 - Too large a d, it may oiscillate

-----------------------------
# MH algorithm :
    
-----------------------------
1. Sample Z<sup>*</sup> ~ J(Z| Z<sup>(s)</sup>)
2. Compute acceptance ratio, r. 
    
r = p(Z<sup>*</sup>|y) / p(Z<sup>(s)</sup>|y) = p(y|Z<sup>*</sup>)p(Z<sup>*</sup>) / p(y|Z<sup>(s)</sup>)p(Z<sup>(s)</sup>)              

3. Z<sup>(s+1)</sup> = 

{  Z<sup>*</sup> probability : min(r,1)

{  Z<sup>(s)</sup> probability : 1 - min(r,1)




In [26]:
y_list = list(data[:,1] + 0.5)
y_list = [math.log(_,math.e) for _ in y_list]
v1 = np.var(y_list)
v2 = np.sum([_*_ for _ in data[:,0]])

In [44]:
var_prior = math.sqrt(v1/v2)

In [45]:
var_prior

0.007227071257966704

In [46]:
num_iterations = 10000

In [93]:
def get_new_beta(old_beta):
    global var_prior
    with pm.Model() as model:
        Z_cov = np.array([
            [var_prior, 0   , 0], 
            [0   , var_prior, 0],
            [0   , 0, var_prior]
        ])
        Z_mu = old_beta
        beta_prior_mv_normal = pm.MvNormal(
            'vals', 
            mu = old_beta, 
            cov = Z_cov,
            shape=[1,3]
        )
    return np.array(beta_prior_mv_normal.random())

In [94]:
beta = np.array([0,0,0])

mu = [0,0,0]
sd = np.array([
    [10, 0, 0],
    [ 0,10, 0],
    [ 0, 0,10]
])

with pm.Model():
    un = pm.Uniform('u',0,1)
list_beta = []

In [ ]:
for _ in range(num_iterations):
    beta_p = get_new_beta(beta)
    # Calculate the acceptance ratio
    p1 = 0
    p2 = 0
    p3 = 0
    p4 = 0
    for x,y in zip(data[:,0],data[:,1]):
        X = np.array([1,x,x*x])
        X_beta_new = np.dot(X,beta_p)
        X_beta_old = np.dot(X,beta)
        with pm.Model():
            dist1 = pm.Poisson('x1', mu=X_beta_new)
            p1 += dist1.logp({'x1':y})
        with pm.Model():
            dist2 = pm.Poisson('x2', mu=X_beta_old)
            p2 += dist2.logp({'x2':y})
            
        with pm.Model():
            dist3 = pm.MvNormal(
                'x3', 
                mu = mu, 
                cov = sd,
                shape=3
            )
            _ = dist3.logp({'x3':beta_p})
            p3 += _
            
        
        with pm.Model():
            dist4 = pm.MvNormal(
                'x4', 
                mu = mu, 
                cov = sd,
                shape=3
            )
            _ = dist4.logp({'x4':beta})
            p4 += _
    r = p1 - p2 + p3 -p4
    if un.random() < r :
        beta = beta_p
        
    list_beta.append(beta)
    
        
            

[-0.03534251  1.16676412]


0.4380987068816343
